In [12]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [3]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod.objective = 'MAR03964'

In [4]:
# modify models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964_PR' # ATP demand in PR
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) # close all exchange reactions
    for index, row in df.iterrows():  
        if row.id in [r.id for r in m.reactions]:
            m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub'])# allow influx for RPE exchange reactions (blood exchange)

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [6]:
def fba_analysis(mod, boundary_dicts, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    conditions = dict()
    ovs = dict() # objective values
    f = dict() # fba fluxes
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 0
        
    with mod as m:
            
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # objective values
            ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
            
            if fba.status == 'optimal':

                # write analysis info / results into dicts

                # uptake / secretion
                uptake_summary = m.summary().uptake_flux
                secretion_summary = m.summary().secretion_flux
                uptake[i] = pd.DataFrame(uptake_summary['flux'])
                secretion[i] = pd.DataFrame(secretion_summary['flux'])
                
                # all fluxes
                f[i] = fba.to_frame()['fluxes']
          
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)  
    
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    
    ovs_df  = pd.concat([pd.DataFrame(boundary_dicts),\
                         pd.DataFrame(boundary_labels),\
                          pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T],axis=1)
    
    # uptake / secretion dfs

    uptake_df = pd.concat(uptake,axis=1)
    secretion_df = pd.concat(secretion,axis=1)
    
    secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in secretion_df.index],\
                               columns=['metabolite'],index= secretion_df.index)
    uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in uptake_df.index],\
                               columns=['metabolite'],index= uptake_df.index)
    
    uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
    secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                     
    # sort uptake / secretion dfs
    uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
    secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('fba_analysis_' + results_name + '_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
        
    return [ovs_df, uptake_df, secretion_df, fluxes_df]

In [7]:
blood_ratio = {df['id'][i]:df['ratio [glucose]/[x]'] [i] for i in range(len(df['id']))}
glucose_level = -2.622342019
# blood_ratio.update((x, y*glucose_level) for x, y in blood_ratio.items())
blood_ratio

{'MAR09034_RPE': 1.0,
 'MAR09063_RPE': 0.12618432385874245,
 'MAR09061_RPE': 0.090869939707149,
 'MAR09067_RPE': 0.05555555555555555,
 'MAR09041_RPE': 0.050818260120585705,
 'MAR09046_RPE': 0.05017226528854436,
 'MAR09068_RPE': 0.0409130060292851,
 'MAR09044_RPE': 0.03574504737295435,
 'MAR09069_RPE': 0.03229974160206718,
 'MAR09040_RPE': 0.026485788113695088,
 'MAR09038_RPE': 0.023040482342807925,
 'MAR09066_RPE': 0.017226528854435832,
 'MAR09064_RPE': 0.016731266149870805,
 'MAR09043_RPE': 0.01309216192937123,
 'MAR09065_RPE': 0.01119724375538329,
 'MAR09045_RPE': 0.009474590869939707,
 'MAR09071_RPE': 0.009323858742463394,
 'MAR09062_RPE': 0.008828596037898364,
 'MAR09042_RPE': 0.005383290267011197,
 'MAR09070_RPE': 0.0032730404823428077,
 'MAR11961_RPE': 0.00014857881136950905,
 'MAR09039_RPE': 0.014233419465977604}

In [8]:
def open_blood_exchange(mod,br, glucose_level):  
    br_updated = br.copy()
    br_updated.update((x, y * glucose_level) for x, y in br_updated.items())
    for key in br:
        mod.reactions.get_by_id(key).bounds = (br_updated[key],1000)# allow influx for RPE exchange reactions (blood exchange)
        
def close_blood_exchange(mod,br):  
    for key in br:
        mod.reactions.get_by_id(key).bounds = (0,1000)# allow influx for RPE exchange reactions (blood exchange)

In [9]:
from src.analysis import create_permutation_dicts
# MAR09048_RPE = oxygen exchange
# MAR09034_RPE = glucose exchange
# MAR09135_RPE = lactate exchange
# MAR04896_PR_RPE =  O2[c_PR] <=> O2[e_RPE_PR]

dark_old_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (2.2)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (2.6)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 4.53','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (1.9)','unlimited']})
dark_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-2.2,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-2.62,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(4.53,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-1.9,-1.9),(-1000,1000)]})

dark_new_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (3.51)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (4.18)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 7.25','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (3.21)','unlimited']})
dark_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-3.51,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-4.18,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(7.25,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-3.21,-3.21),(-1000,1000)]})

light_old_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (1.84)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (1.37)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 2.81','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (1.59)','unlimited']})
light_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-1.84,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-1.37,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(2.81,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-1.59,-1.59),(-1000,1000)]})

light_new_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (2.61)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (1.93)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 3.98','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (2.31)','unlimited']})
light_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-2.61,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-1.93,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(3.98,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-2.31,-2.31),(-1000,1000)]})

OLD
dark	glucose (in)	pmol/s/mm2 	2.62
light	glucose (in)	pmol/s/mm2 	1.37

NEW
dark	glucose (in)	pmol/s/mm2 	4.18
light	glucose (in)	pmol/s/mm2 	1.93

In [ ]:
# mod_RPE_PR
open_blood_exchange(mod_RPE_PR,blood_ratio,-2.62)
dark_old_CS = fba_analysis(mod_RPE_PR, dark_old_estimates, dark_old_labels, 'CS_dark_old_ox2.2')
light_old_CS = fba_analysis(mod_RPE_PR, light_old_estimates, light_old_labels, 'CS_light_old_ox1.84')
open_blood_exchange(mod_RPE_PR,blood_ratio,-4.18)            
dark_new_CS = fba_analysis(mod_RPE_PR, dark_new_estimates, dark_new_labels, 'CS_dark_new_ox3.51')
light_new_CS = fba_analysis(mod_RPE_PR, light_new_estimates, light_new_labels, 'CS_light_new_ox2.61')

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\u

KeyboardInterrupt: 

In [11]:
# mod_Human1_Human1
open_blood_exchange(mod_Human1_Human1,blood_ratio,-2.62)
dark_old_H1 = fba_analysis(mod_Human1_Human1, dark_old_estimates, dark_old_labels, 'H1_dark_old_ox2.2')
light_old_H1 = fba_analysis(mod_Human1_Human1, light_old_estimates, light_old_labels, 'H1_light_old_ox1.84')
open_blood_exchange(mod_Human1_Human1,blood_ratio,-4.18)     
dark_new_H1 = fba_analysis(mod_Human1_Human1, dark_new_estimates, dark_new_labels, 'H1_dark_new_ox3.51')
light_new_H1 = fba_analysis(mod_Human1_Human1, light_new_estimates, light_new_labels, 'H1_light_new_ox2.61')

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\u

In [ ]:
stop

In [ ]:
close_blood_exchange(mod_RPE_PR,blood_ratio)

In [ ]:
from src.analysis import create_permutation_dicts
dict_all_blood_exchange_1 = {df_id:(-1,1000) for df_id in list(df['id'])}
dict_each_blood_exchange_1 = [{df_id:(-1,1000)} for df_id in list(df['id'])]
dict_each_blood_exchange_1

dict_oxygen = create_permutation_dicts({'MAR09048_RPE':[(0,0),(-2.2,1000),(-2.61,1000),(-1000,1000)]})
dict_oxygen
dict_each_blood_exchange_1

In [ ]:
dict_all_blood_exchange_1 = {df_id:(-1,1000) for df_id in list(df['id'])}
dict_all_blood_exchange_1_oxygen = create_permutation_dicts(\
                        {'MAR09048_RPE':[(0,0),(-2.2,1000),(-2.61,1000),(-1000,1000)],
                         'MAR09034_RPE': [(-1, 1000)],
                         'MAR09135_RPE': [(-1, 1000)],
                         'MAR09047_RPE': [(-1, 1000)],
                         'MAR09063_RPE': [(-1, 1000)],
                         'MAR09061_RPE': [(-1, 1000)],
                         'MAR09067_RPE': [(-1, 1000)],
                         'MAR09041_RPE': [(-1, 1000)],
                         'MAR09046_RPE': [(-1, 1000)],
                         'MAR09068_RPE': [(-1, 1000)],
                         'MAR09044_RPE': [(-1, 1000)],
                         'MAR09069_RPE': [(-1, 1000)],
                         'MAR09040_RPE': [(-1, 1000)],
                         'MAR09038_RPE': [(-1, 1000)],
                         'MAR09066_RPE': [(-1, 1000)],
                         'MAR09064_RPE': [(-1, 1000)],
                         'MAR09043_RPE': [(-1, 1000)],
                         'MAR09065_RPE': [(-1, 1000)],
                         'MAR09045_RPE': [(-1, 1000)],
                         'MAR09071_RPE': [(-1, 1000)],
                         'MAR09062_RPE': [(-1, 1000)],
                         'MAR09042_RPE': [(-1, 1000)],
                         'MAR09070_RPE': [(-1, 1000)],
                         'MAR11961_RPE': [(-1, 1000)],
                         'MAR09039_RPE': [(-1, 1000)]})

In [ ]:
dict_each_blood_exchange_1_oxygen = \
                [{'MAR09034_RPE': (-1, 1000)},
                 {'MAR09048_RPE': (-1, 1000)},
                 {'MAR09135_RPE': (-1, 1000)},
                 {'MAR09047_RPE': (-1, 1000)},
                 {'MAR09063_RPE': (-1, 1000)},
                 {'MAR09061_RPE': (-1, 1000)},
                 {'MAR09067_RPE': (-1, 1000)},
                 {'MAR09041_RPE': (-1, 1000)},
                 {'MAR09046_RPE': (-1, 1000)},
                 {'MAR09068_RPE': (-1, 1000)},
                 {'MAR09044_RPE': (-1, 1000)},
                 {'MAR09069_RPE': (-1, 1000)},
                 {'MAR09040_RPE': (-1, 1000)},
                 {'MAR09038_RPE': (-1, 1000)},
                 {'MAR09066_RPE': (-1, 1000)},
                 {'MAR09064_RPE': (-1, 1000)},
                 {'MAR09043_RPE': (-1, 1000)},
                 {'MAR09065_RPE': (-1, 1000)},
                 {'MAR09045_RPE': (-1, 1000)},
                 {'MAR09071_RPE': (-1, 1000)},
                 {'MAR09062_RPE': (-1, 1000)},
                 {'MAR09042_RPE': (-1, 1000)},
                 {'MAR09070_RPE': (-1, 1000)},
                 {'MAR11961_RPE': (-1, 1000)},
                 {'MAR09039_RPE': (-1, 1000)}]

In [ ]:
def fba_analysis_exchange(mod, boundary_dicts, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    conditions = dict()
    ovs = dict() # objective values
    f = dict() # fba fluxes
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 0
        
    with mod as m:
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # objective values
            ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
            
            if fba.status == 'optimal':

                # write analysis info / results into dicts

                # uptake / secretion
                uptake_summary = m.summary().uptake_flux
                secretion_summary = m.summary().secretion_flux
                uptake[i] = pd.DataFrame(uptake_summary['flux'])
                secretion[i] = pd.DataFrame(secretion_summary['flux'])
                
                # all fluxes
                f[i] = fba.to_frame()['fluxes']
          
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)  
    
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    
    ovs_df  = pd.concat([pd.DataFrame(boundary_dicts),\
                         pd.DataFrame(boundary_labels),\
                          pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T],axis=1)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
    secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('fba_analysis_' + results_name + '_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
        
    return [ovs_df, uptake_df, secretion_df, fluxes_df]

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,1000) # open reaction
                m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
                fba = m.optimize()
                results_dict[row['id']] = fba.objective_value
                m.reactions.get_by_id(row['id']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000)) # anearobic
r2 = ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000)) # limited oxygen
r3 = ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
r4 = ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000)) # anearobic
r5 = ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
r6 = ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

df_results = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values(),r4.values(),r5.values(),r6.values()]).transpose()
df_results.to_clipboard()
df_results

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],1000) # open reaction
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR00611_RPE').bounds = (-1,1000)
        fba = m.optimize()
        print(fba.objective_value)
    return 

ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],1000)# open reaction
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        mod_RPE_PR.reactions.get_by_id('MAR00611_RPE').bounds = (0,1000)
        fba = m.optimize()
        print(fba.objective_value)
    return 

ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],1000) # open reaction
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        mod_RPE_PR.reactions.get_by_id('MAR09209_RPE').bounds = (-10,1000)
        fba = m.optimize()
        print(fba.objective_value)
    return 

ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000)) # anearobic
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

In [ ]:
# ATP yield met exchange allowed (including glucose)
def ATP_yield_analysis(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield met exchange allowed (including glucose) + LACTATE unlimited
def ATP_yield_analysis(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1000,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield only glucose exchange allowed
def ATP_yield_analysis_only_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        m.reactions.get_by_id('MAR09034_RPE').bounds = (-2.622342019,1000) # glucose
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# TEST
def ATP_yield_analysis_only_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,0)
        rxns = [r for r in m.reactions if '_RPE' in r.id if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,row['ub']) # blood exchange
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1,1000) # LACTATE 
        m.reactions.get_by_id('MAR09034_RPE').bounds = (-1,1000) # glucose
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield only glucose exchange allowed + LACTATE UNLIMITED
def ATP_yield_analysis_only_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        m.reactions.get_by_id('MAR09034_RPE').bounds = (-2.622342019,1000) # glucose
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1000,1000) # LACTATE # MAR09135_RPE
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield met exchange allowed, but no glucose allowed  + LACTATE UNLIMTIED
def ATP_yield_analysis_no_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1000,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield met exchange allowed, but no glucose allowed
def ATP_yield_analysis_no_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield met exchange allowed (high!), but no glucose allowed   + LACTATE UNLIMTIED
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1000,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield met exchange allowed (high!), but no glucose allowed
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP  no glucose, mets allowed, lactate -1
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135_RPE').bounds = (-1,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP  no glucose,amino acids (high flux; -1,1000), lactate efflux
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135_RPE').bounds = (4.544294699,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_RPE_PR, (-1000,1000)))

print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod_Human1_Human1, (-1000,1000)))

In [ ]:
# ATP yield per metabolite (low metabolite influx)
def ATP_yield_analysis_per_aa_low_influx(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open reaction
                m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
                fba = m.optimize()
                results_dict[row['id']] = fba.objective_value
                m.reactions.get_by_id(row['id']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa_low_influx(mod_RPE_PR,(0,1000)) # anearobic
r2 = ATP_yield_analysis_per_aa_low_influx(mod_RPE_PR,(-2.2,1000)) # limited oxygen
r3 = ATP_yield_analysis_per_aa_low_influx(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
r4 = ATP_yield_analysis_per_aa_low_influx(mod_Human1_Human1,(0,1000)) # anearobic
r5 = ATP_yield_analysis_per_aa_low_influx(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
r6 = ATP_yield_analysis_per_aa_low_influx(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

df_results2 = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values(),r4.values(),r5.values(),r6.values()]).transpose()
df_results2.to_clipboard()
df_results2

In [ ]:
# ATP yield, no glucose, low aa exchange
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub']) # open reaction
                m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose bounds
        
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000))) # unlimited oxyen

print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000))) # unlimited oxyen

In [ ]:
# ATP yield, no glucose, high aa exchange
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        m = close_EX(m)
        m = open_RPE_EX_ub(m)
        for index, row in df.iterrows():  
            if row.id in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id']).bounds = (-1,1000) # open reaction
                m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034_RPE').bounds = (0,1000) # glucose bounds
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_per_aa(mod_RPE_PR,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_RPE_PR,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_RPE_PR,(-1000,1000))) # unlimited oxyen

print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod_Human1_Human1,(-1000,1000))) # unlimited oxyen

In [ ]:
# ATP yield per metabolite (low metabolite influx)
def ATP_yield_analysis_per_met(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,0) # close all exchange
        rxns = [r for r in m.reactions if '_RPE' in r.id if len(r.products) == 0]
        for r in rxns:
            r.bounds=(-1,1000) # open RPE exchange one by one
            m.reactions.get_by_id('MAR09048_RPE').bounds = oxygen_bounds # OXYGEN
            fba = m.optimize()
            results_dict[r.id] = fba.objective_value
            r.bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_met(mod_RPE_PR,(0,1000)) # anearobic
r2 = ATP_yield_analysis_per_met(mod_RPE_PR,(-2.2,1000)) # limited oxygen
r3 = ATP_yield_analysis_per_met(mod_RPE_PR,(-1000,1000)) # unlimited oxyen
r4 = ATP_yield_analysis_per_met(mod_Human1_Human1,(0,1000)) # anearobic
r5 = ATP_yield_analysis_per_met(mod_Human1_Human1,(-2.2,1000)) # limited oxygen
r6 = ATP_yield_analysis_per_met(mod_Human1_Human1,(-1000,1000)) # unlimited oxyen

df_results2 = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values(),r4.values(),r5.values(),r6.values()]).transpose()
df_results2.to_clipboard()
df_results2

In [ ]:
from src.analysis import fba_analysis, create_permutation_dicts                                
fba1 = fba_analysis(mod_RPE_PR,lipids_exchange_dict,'MAR03964_PR',)
fba2 = fba_analysis(mod_Human1_Human1,lipids_exchange_dict,'MAR03964_PR')

In [ ]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model.copy() as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion


In [ ]:
    boundary_dicts = Human1_dict_p
    objective = 'MAR03964_PR'
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with mod_RPE_PR as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]

In [ ]:
from src.get_info import make_rxn_df
df = make_rxn_df(mod_RPE_PR)
DF2 = df[df.index.isin([r.id for r in mod_RPE_PR.reactions if len(r.products)==0])]
DF2.to_clipboard()

In [ ]:
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)
    m.reactions.get_by_id('MAR09048_RPE').bounds=(-1000,1000)
    m.reactions.get_by_id('MAR09034_RPE').bounds=(-10,1000)
    m.reactions.get_by_id('MAR03964_RPE').bounds = (40,40)
    m.objective = 'MAR03964_PR'

In [ ]:
mod_Human1_Human1.optimize()

In [ ]:
mod_RPE_PR.optimize()

In [ ]:
mod_RPE_PR_old.optimize()

In [ ]:
mod_RPE_PR.reactions.get_by_id('MAR10023_RPE_PR')

In [ ]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Recon3D_dict = {'EX_o2s[e]_RPE':[(-10,1000)],'EX_glc_D[e]_RPE':[(-30,1000)],'DM_atp_c__RPE':[(0,0),(40,40),(80,80)]}
mitocore_dict = {'EX_o2_e':[(-1000,1000)],'EX_glc_D_e':[(-300,1000)]}

Human1_dict_p = create_permutation_dicts(Human1_dict)
Recon3D_dict_p = create_permutation_dicts(Recon3D_dict)
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

In [ ]:
from src.modify_model import change_bounds
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE').bounds = (-10,1000)
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (-100,1000)

In [ ]:
df = make_rxn_df(mod_RPE_PR)

In [ ]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [ ]:
def fba_fva_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            fva = flux_variability_analysis(m, loopless=False) #fva

            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            f_min[i] = fva['minimum']
            f_max[i] = fva['minimum']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000),(-40,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max, uptake, secretion = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
Human1_dict = create_permutation_dicts({'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0)]})
fba_fva_analysis(mod_RPE_PR, Human1_dict, 'MAR03964_PR')

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Human1_dict_p = create_permutation_dicts(Human1_dict)

fba_fva_analysis(mod_RPE_PR,Human1_dict_p, 'MAR03964_PR')